In [ ]:
import wandb
run = wandb.init(project="my-project")

In [ ]:
import os, json 
from research.utils.data_access_utils import S3AccessUtils

def load_model(key, tag, model_url, is_url):
    s3 = S3AccessUtils('/root/data', json.load(open(os.environ['AWS_CREDENTIALS'])))
    weight_model_f, _, _ = s3.download_from_url(model_url)

    dataset_artifact = wandb.Artifact(tag, type = 'single_fish_model', metadata = {
        'tag': tag,
        'url': model_url,
        'is_url': is_url
    })

    dataset_artifact.add_file(weight_model_f, 'model')

    wandb.log_artifact(dataset_artifact)
    
models_to_load = [
    ('weight_v1', 'curr-nonsynthetic', 'https://aquabyte-models.s3-us-west-1.amazonaws.com/biomass/playground/nn_epoch_798_v2.pb', True)
]

key, tag, model_url, is_url = models_to_load[0]

load_model(key, tag, model_url, is_url)

In [ ]:

    
"""This module contains utility helper functions for the WeightEstimator class."""

from collections import namedtuple
from typing import Dict, List, Tuple
import numpy as np
import torch
from research.weight_estimation.keypoint_utils import body_parts


CameraMetadata = namedtuple('CameraMetadata',
                            ['focal_length', 'focal_length_pixel', 'baseline_m',
                             'pixel_count_width', 'pixel_count_height', 'image_sensor_width',
                             'image_sensor_height'])


def get_left_right_keypoint_arrs(annotation: Dict[str, List[Dict]]) -> Tuple:
    """Gets numpy array of left and right keypoints given input keypoint annotation.
    Args:
        annotation: dict with keys 'leftCrop' and 'rightCrop'. Values are lists where each element
        is a dict with keys 'keypointType', 'xCrop' (num pixels from crop left edge),
        'yCrop' (num pixels from crop top edge), 'xFrame' (num pixels from full frame left edge),
        and 'yFrame' (num pixels from full frame top edge).
    Returns:
        X_left: numpy array containing left crop (xFrame, yFrame) for each key-point ordered
        alphabetically.
        X_right: same as above, but for right crop.
    """

    left_keypoints, right_keypoints = {}, {}
    for item in annotation['leftCrop']:
        body_part = item['keypointType']
        left_keypoints[body_part] = (item['xFrame'], item['yFrame'])

    for item in annotation['rightCrop']:
        body_part = item['keypointType']
        right_keypoints[body_part] = (item['xFrame'], item['yFrame'])

    left_keypoint_arr, right_keypoint_arr = [], []
    for body_part in body_parts.core_body_parts:
        left_keypoint_arr.append(left_keypoints[body_part])
        right_keypoint_arr.append(right_keypoints[body_part])

    X_left = np.array(left_keypoint_arr)
    X_right = np.array(right_keypoint_arr)
    return X_left, X_right


def normalize_left_right_keypoint_arrs(X_left: np.ndarray, X_right: np.ndarray) -> Tuple:
    """Normalizes input left and right key-point arrays. The normalization involves (1) 2D
    translation of all keypoints such that they are centered, (2) rotation of the 2D coordiantes
    about the center such that the line passing through UPPER_LIP and fish center is horizontal.
    """

    # translate key-points, perform reflection if necessary
    upper_lip_idx = body_parts.core_body_parts.index(body_parts.UPPER_LIP)
    tail_notch_idx = body_parts.core_body_parts.index(body_parts.TAIL_NOTCH)
    if X_left[upper_lip_idx, 0] > X_left[tail_notch_idx, 0]:
        X_center = 0.5 * (np.max(X_left, axis=0) + np.min(X_left, axis=0))
        X_left_centered = X_left - X_center
        X_right_centered = X_right - X_center
    else:
        X_center = 0.5 * (np.max(X_right, axis=0) + np.min(X_right, axis=0))
        X_left_centered = X_right - X_center
        X_right_centered = X_left - X_center
        X_left_centered[:, 0] = -X_left_centered[:, 0]
        X_right_centered[:, 0] = -X_right_centered[:, 0]

    # rotate key-points
    upper_lip_x, upper_lip_y = tuple(X_left_centered[upper_lip_idx])
    theta = np.arctan(upper_lip_y / upper_lip_x)
    R = np.array([
        [np.cos(theta), -np.sin(theta)],
        [np.sin(theta), np.cos(theta)]
    ])

    D = X_left_centered - X_right_centered
    X_left_rot = np.dot(X_left_centered, R)
    X_right_rot = X_left_rot - D
    return X_left_rot, X_right_rot


def convert_to_world_point_arr(X_left: np.ndarray, X_right: np.ndarray,
                               camera_metadata: CameraMetadata) -> np.ndarray:
    """Converts input left and right normalized keypoint arrays into world coordinate array."""

    y_world = camera_metadata.focal_length_pixel * camera_metadata.baseline_m / \
              (X_left[:, 0] - X_right[:, 0])

    # Note: the lines commented out below are technically the correct formula for conversion
    # x_world = X_left[:, 0] * y_world / camera_metadata.focal_length_pixel
    # z_world = -X_left[:, 1] * y_world / camera_metadata.focal_length_pixel
    x_world = ((X_left[:, 0] * camera_metadata.image_sensor_width / camera_metadata.pixel_count_width) * y_world) / (camera_metadata.focal_length)
    z_world = (-(X_left[:, 1] * camera_metadata.image_sensor_height / camera_metadata.pixel_count_height) * y_world) / (camera_metadata.focal_length)
    X_world = np.vstack([x_world, y_world, z_world]).T
    return X_world


def stabilize_keypoints(X: np.ndarray) -> np.ndarray:
    """Transforms world coordinate array so that neural network inputs are stabilized"""
    X_new = np.zeros(X.shape)
    X_new[:, 0] = 0.5 * X[:, 0] / X[:, 1]
    X_new[:, 1] = 0.5 * X[:, 2] / X[:, 1]
    X_new[:, 2] = 0.05 / X[:, 1]
    return X_new


def convert_to_nn_input(annotation: Dict[str, List[Dict]], camera_metadata: CameraMetadata) \
        -> torch.Tensor:
    """Convrts input keypoint annotation and camera metadata into neural network tensor input."""
    X_left, X_right = get_left_right_keypoint_arrs(annotation)
    X_left_norm, X_right_norm = normalize_left_right_keypoint_arrs(X_left, X_right)
    X_world = convert_to_world_point_arr(X_left_norm, X_right_norm, camera_metadata)
    X = stabilize_keypoints(X_world)
    nn_input = torch.from_numpy(np.array([X])).float()
    return nn_input


"""
This module contains the WeightEstimator class for estimating fish weight (g), length (mm), and
k-factor given input keypoint coordinates and camera metadata.
"""

from typing import Dict, Tuple
import torch
from torch import nn


class Network(nn.Module):
    """Network class defines neural-network architecture for both weight and k-factor estimation
    (currently both neural networks share identical architecture)."""

    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(24, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        """Run inference on input keypoint tensor."""
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.output(x)
        return x


class WeightEstimator:
    """WeightEstimator class is used to predict fish weight, k-factor, and length
    given input keypoint annotations and camera metadata."""

    def __init__(self, weight_model_f: str, kf_model_f: str) -> None:
        """Initializes class with input weight and k-factor neural-networks."""
        self.weight_model = Network()
        self.weight_model.load_state_dict(torch.load(weight_model_f))
        self.weight_model.eval()

        self.kf_model = Network()
        self.kf_model.load_state_dict(torch.load(kf_model_f))
        self.kf_model.eval()

    @staticmethod
    def _get_model_input(annotation: Dict, camera_metadata: CameraMetadata) -> torch.Tensor:
        """Generates neural-network input tensor given annotation and camera_metadata."""
        X = convert_to_nn_input(annotation, camera_metadata)
        return X

    def predict_weight(self, annotation: Dict, camera_metadata: CameraMetadata) -> float:
        """Generates weight prediction given input annotation and camera metadata."""
        X = self._get_model_input(annotation, camera_metadata)
        weight = 1e4 * self.weight_model(X).item()
        return weight

    def predict_kf(self, annotation: Dict, camera_metadata: CameraMetadata) -> float:
        """Generates k-factor prediction gievn input annotation and camera metadata."""
        X = self._get_model_input(annotation, camera_metadata)
        kf = self.kf_model(X).item()
        return kf

    def predict(self, annotation: Dict, camera_metadata: CameraMetadata) -> Tuple:
        """Generates weight, k-factor, and length predictions given input annotation and camera
        metadata."""
        weight = self.predict_weight(annotation, camera_metadata)
        kf = self.predict_kf(annotation, camera_metadata)
        if weight * kf > 0:
            length = (1e5 * weight / kf) ** (1.0 / 3)
        else:
            length = 0
        return weight, length, kf

In [ ]:
def get_dataset(dataset_name):
    dataset_artifact = run.use_artifact(datasets[0])
    dataset_metadata = dataset_artifact.metadata
    table = dataset_artifact.get('biomass_estimations')
    df = pd.DataFrame(table.data, columns = table.columns)
    
    return { 'df': df, 'name': dataset_name, 'metadata': metadata }

def run_single_fish_model(dataset, model_name):
    model_artifact = run.use_artifact(model_name)
    model = model_artifact.get_path('model')
    weight_model_f = model.download()
    
    run = wandb.init(project="my-project", job_type='single_fish_run', config={
        'model_name': model_name,
        'dataset_name': dataset['name']
    })

    kf_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/k-factor/playground/kf_predictor_v2.pb')

    weight_estimator = WeightEstimator(weight_model_f, kf_model_f)

    weights = []

    for idx, row in dataset['df'].iterrows():
        annotation = row.annotation
        if not annotation:
            weights.append(None)
            continue
        camera_metadata = row.camera_metadata

        camera_metadata_obj = CameraMetadata(
            focal_length=camera_metadata['focalLength'],
            focal_length_pixel=camera_metadata['focalLengthPixel'],
            baseline_m=camera_metadata['baseline'],
            pixel_count_width=camera_metadata['pixelCountWidth'],
            pixel_count_height=camera_metadata['pixelCountHeight'],
            image_sensor_width=camera_metadata['imageSensorWidth'],
            image_sensor_height=camera_metadata['imageSensorHeight']
        )

        weight, length, kf = weight_estimator.predict(annotation, camera_metadata_obj)
        weights.append(weight)
    
    weight_run_artifact = wandb.Artifact('first_weight_run', type = 'weight_run', metadata = {
        'model_name': model_name,
        'dataset_name': dataset['name']
    })

    table = wandb.Table(dataframe=df)
    
    weight_run_artifact.add(table, 'weights')

    wandb.log_artifact(weight_run_artifact)

    return weights

def generate_raw_individual_values(pm_base, gt_metadata, start_hour, end_hour, apply_growth_rate, max_day_diff, days_post_feeding, final_days_post_feeding):
    last_feeding_date = gt_metadata['last_feeding_date']
    date = add_days(last_feeding_date, days_post_feeding)
    weights, _ = generate_smart_individual_values(pm_base, date, max_day_diff, True, apply_growth_rate, 0.9)
    return weights


def generate_average_weight_accuracy(weights, gt_metadata, loss_factor):
    avg_weight_prediction = np.mean(weights)
    gutted_weight_prediction = avg_weight_prediction * (1.0 - loss_factor)
    gt_weight = gt_metadata['gutted_average_weight']
    avg_weight_err = (gutted_weight_prediction - gt_weight) / gt_weight
    return avg_weight_err, gutted_weight_prediction

def generate_distribution_accuracy(weights, gt_metadata, loss_factor):
    gutted_weights = weights * (1.0 - loss_factor)
    gutted_weight_distribution = gt_metadata['gutted_weight_distribution']
    
    if gutted_weight_distribution is None:
        return []
    
    count_distribution_errors = []
    
    for bucket in gutted_weight_distribution:
        lower_bound, upper_bound = bucket.split('-')
        pct = gutted_weight_distribution[bucket]
        mask = (gutted_weights >= float(lower_bound) * 1000) & (gutted_weights < float(upper_bound) * 1000)

        pct = np.sum(mask) / len(mask)
        gt_pct = gutted_weight_distribution[bucket] / 100
        
        count_distribution_errors.append(pct - gt_pct)
        
    return count_distribution_errors

In [ ]:
datasets = [
    'bryton/my-project/LAP2_20200730_:v0',
    'bryton/my-project/BKJ03_20210206:latest'
]

models = [
    'bryton/my-project/curr-nonsynthetic:latest'
]

In [ ]:
dataset_name = datasets[0]
model_name = models[0]

dataset = get_dataset(dataset_name)

df = dataset['df']

df['estimated_weight_g_new'] = get_weights(dataset, model_name)



In [ ]:
def get_start_end_hour():
    if method == 'manual':
        for start_hour in start_hours:
            for end_hour in end_hours:
                start_end_hours.append((method, start_hour, end_hour))
    elif method == 'u-shape':
        df2 = df[(df.hour >= 3) & (df.hour <= 20)]

        #count, bins, _ = plt.hist(df2.hour, density = True, bins = (np.max(df2.hour) - np.min(df2.hour)))

        start_hour = np.min(df2.hour)
        end_hour = np.max(df2.hour)

        bins = np.arange(start_hour, end_hour + 1)

        weights = []

        for hour in np.arange(start_hour, end_hour + 1):
            avg_weight = np.mean(df2[df2.hour == hour].estimated_weight_g)
            weights.append(avg_weight)

        start_index = np.where(bins == 10)[0][0]

        lower_index = start_index
        upper_index = start_index

        is_iterating = True
        eps = 3

        while is_iterating:
        #     print(np.std(weights[lower_index:upper_index]))
            if lower_index > 0 and upper_index < len(weights) - 1 and np.abs(weights[upper_index + 1] - weights[lower_index - 1]) < eps * np.std(weights[lower_index - 1:upper_index + 1]):
                lower_index = lower_index - 1
                upper_index = upper_index + 1
            elif lower_index > 0 and np.abs(weights[upper_index] - weights[lower_index - 1]) < eps * np.std(weights[lower_index - 1:upper_index]):
                lower_index = lower_index - 1
            elif upper_index < len(weights) - 1 and np.abs(weights[upper_index + 1] - weights[lower_index]) < eps * np.std(weights[lower_index:upper_index + 1]):
                upper_index = upper_index + 1
            else:
                is_iterating = False

        start_hour, end_hour = bins[lower_index], bins[upper_index]

        start_end_hours.append((method, start_hour, end_hour))
    elif method == 'hour_hist':
        df2 = df[(df.hour >= 3) & (df.hour <= 20)]

        count, bins, _ = plt.hist(df2.hour, density = True, bins = (np.max(df2.hour) - np.min(df2.hour)))

        idx_values = np.where(count > 1.0 / 18)[0]

        start_index = np.where(bins == 10)[0][0]
        start_array = np.where(idx_values == start_index)[0][0]

        lower_index = start_array
        upper_index = start_array

        while lower_index > 0 and (idx_values[lower_index] - idx_values[lower_index - 1] == 1):
            lower_index = lower_index - 1
        while upper_index < len(idx_values) - 1 and (idx_values[upper_index + 1] - idx_values[upper_index] == 1):
            upper_index = upper_index + 1

        start_hour, end_hour = bins[idx_values[lower_index]], bins[idx_values[upper_index]]

        start_end_hours.append((method, start_hour, end_hour))

In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt
from research.utils.datetime_utils import add_days
from report_generation.report_generator import generate_ts_data, SamplingFilter
from report_generation.report_generator import gen_pm_base
from population_metrics.smart_metrics import generate_smart_avg_weight, generate_smart_individual_values, ValidationError

start_hours = [7]
end_hours = [15]
apply_growth_rate = True
max_day_diff = 3
days_post_feeding = 1
final_days_post_feeding = 1
loss_factors = [0.16, 'expected_loss_factor'] # need to determine the right values here
akpd_cutoffs = [0.01, 0.95]

hour_filter_methods = ['manual', 'hour_hist', 'u-shape'] #  'u-shape',

for loss_factor in loss_factors:
    avg_weight_error_col.append([])
    avg_weight_error_col.append([])
    count_distribution_error_col.append([])

gt_metadata = meta
last_feeding_date = gt_metadata['last_feeding_date']
slaughter_date = gt_metadata['slaughter_date']

if slaughter_date is not None and last_feeding_date is not None:
    date_diff = datetime.strptime(slaughter_date, '%Y-%m-%d') - datetime.strptime(last_feeding_date, '%Y-%m-%d')
    starvation_days = date_diff.days
else:
    starvation_days = None

df['estimated_weight_g'] = df['estimated_weight_g_new']
final_date_post_feeding = add_days(gt_metadata['last_feeding_date'], final_days_post_feeding)
tdf = df[df.date <= final_date_post_feeding]

start_end_hours = []

for method in hour_filter_methods:
    

for akpd_cutoff in akpd_cutoffs:
    for method, start_hour, end_hour in start_end_hours:
        sampling_filter = SamplingFilter(
            start_hour=start_hour,
            end_hour=end_hour,
            kf_cutoff=0.0
        )

        pm_base = gen_pm_base(tdf, sampling_filter)

        try:
            weights = generate_raw_individual_values(pm_base, gt_metadata, start_hour, end_hour, apply_growth_rate, max_day_diff, days_post_feeding, final_days_post_feeding)
        except ValidationError as err:
            continue

        for index, loss_factor in enumerate(loss_factors):
            if loss_factor == 'expected_loss_factor':
                loss_factor = gt_metadata['expected_loss_factor'] or 0.165

                if loss_factor > 10:
                    loss_factor = loss_factor / 100.0

            avg_weight_err, gutted_weight_prediction = generate_average_weight_accuracy(weights, gt_metadata, loss_factor)
            avg_weight_error_col[index].append(avg_weight_err)

            count_distribution_errors = generate_distribution_accuracy(weights, gt_metadata, loss_factor)
            count_distribution_error_col[index].append(count_distribution_errors)
            
            config = {
                'model_name': 'bryton/my-project/curr-nonsynthetic:latest',
                'dataset_name': 'bryton/my-project/BKJ03_20210206:latest',
                'weight_run': 'bryton/my-project/first_weight_run:v0',

                'hour_filter_method': method,
                'akpd_cutoff': akpd_cutoff,
                'start_hour': start_hour,
                'end_hour': end_hour,
                'loss_factor': loss_factor,
                'starvation_days': starvation_days,
                'avg_weight': np.mean(weights),
                'gt_avg_weight': gt_metadata['gutted_average_weight'],
                'avg_weight_err': avg_weight_err,
                'gutted_weight_prediction': gutted_weight_prediction,
                'count_distribution_errors': count_distribution_errors,

                'all_weights_raw': df['estimated_weight_g'].values,
                'used_weights': wandb.Histogram(weights)
            }
            config.update(meta)

            wandb.init(project="my-project", config=config)
            
            data = [[s] for s in weights]
            table = wandb.Table(data=data, columns=["weights"])
            wandb.log({ 'histogram': wandb.plot.histogram(table, 'weights') })


In [ ]:
gt = pd.read_csv('eide_langoy_singleweights.csv')
langoy_gt = gt['weight'] * 1000 / 0.84

pen5 = pd.read_csv('blom_vikane_singleweights.csv')
vikane_gt = pen5['weight'] * 1000 / 0.84

# imr_gt = pd.read_csv('imr.csv').weight

single_weights = [
    ('langoy_pen_id_108_2020-05-07_2020-05-17', langoy_gt),
    ('vikane_pen_id_60_2020-08-10_2020-08-30', vikane_gt)
]